# Anger Camera SPICE-Integrated Data Reduction Script
***

This reduction and analysis notebook was created for the HFIR cycle 503 spin-energy entanglement experiment with the IU Rf flippers: 8/29 - 9/22 (2023) at CG4B.

To do list:
1. Modify code to work with time-of-flight
2. Fit each pixel to a cosine curve
3. Write Fourier analysis code
4. Witness calculation

## Defining various useful functions and constants

In [2]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy.integrate import simpson
from os.path import isdir,isfile

laptop_direct = "C:/Users/samck/Desktop/Anger_data/"
desktop_direct = "C:/Users/xsm/Desktop/MIEZE_data/"
DEFAULT_DIRECT = desktop_direct  #can also be changed in function calls
assert isdir(DEFAULT_DIRECT), 'Default directory is not found!'
print(f'Default directory: {DEFAULT_DIRECT}')

SMALL_SIZE,MEDIUM_SIZE,BIGGER_SIZE = 20,25,35
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('axes', titlesize=BIGGER_SIZE)    # actually gets title font size (glitch in matplotlib?)
plt.rcParams.update({'axes.grid':True})

pixels_to_mm = 116./512.  #0.227 mm per pixel in Anger camera
CL = -4.63  #larmor constant (in Gauss and Angstroms)
LAM = 5.5  #CG4B wavelength (0.1% delta lambda)
GPA = 16  #Rf flipper and phase coil Gauss/amps
delta_RF = 8.5  #RF flipper separation (cm)

def cos(x,*args,**kwargs):
    """Returns a cosine with parameters determined by ().
    Standard argument list is ampitude,frequency,phase,shim."""
    assert len(args) + len(kwargs) == 4, 'Must specify 4 parameters: amp, freq, phase, and shim!'
    i=0
    if 'amp' in kwargs: amp = kwargs.get('amp')
    else:
        amp = args[i]
        i += 1
    if 'freq' in kwargs: freq = kwargs.get('freq')
    else:
        freq = args[i]
        i += 1
    if 'phase' in kwargs: phase = kwargs.get('phase')
    else: 
        phase = args[i]
        i += 1
    if 'shim' in kwargs: shim = kwargs.get('shim')
    else: 
        shim = args[i]
    return amp*np.cos(freq*x + phase) + shim

def my_cos(x,amp,freq,phase):
    """Offset given by center guide current start current."""
    return amp*np.cos(freq*(x - .5) + phase)

def get_flip(dist,gpa=30.,lam=LAM,flip=np.pi,show=True):
    """Returns the current required for pi or pi/2 flip."""
    result = round(flip/(cl*gpa*lam*dist*1e-2),2)
    if show: print(f'Current required for a {round(flip,2)} flip: {result} amps')
    return result

def frp_convert(value,show=False):
    """Convert polarization to flipping ratio and vice-versa."""
    assert value != 1, 'FR is infinite when polarization is unity!'
    if value > 1:
        result = (value - 1)/(value + 1)
        something = f'FR: {round(value,3)} \n Pol: {round(result*100,1)}%'
    else:
        result = (1 + value)/(1 - value)
        something = f'FR: {round(result,3)} \n Pol: {round(value*100,1)}%'
    if show: print(something)
    return result

def get_res(D,Ls,Ld,show=True):
    """Calculates geometric resolution."""
    res = D*Ls/Ld
    if show: print(f'Geometric resolution: {round(res,3)} (mm) \n')
    return res

def get_focus(f1,f2,delta=delta_RF,show=False):
    """Returns focusing condition for flippers in MIEZE-mode.
    Frequency is assumed to be in kHz."""
    assert f2 > f1, 'f1 > f2!'
    result = delta/2*(f2 + f1)/(f2 - f1)
    if show:
        print(f'Detector distance: {round(result,2)} cm')
        print(f'MIEZE frequency: {round(f2-f1,2)} kHz')
        print(f'Modulation frequency: {round(2*np.pi*(f2-f1),2)} rad/s')
    return result

def degauss_timer(start,delta_I=2,ramp_rate=.1,offset=2):
    """Estimates the time for a degauss."""
    steps = start
    result = steps*ramp_rate + 
    print(f'Estimated degauss time: {round(result/60.)} min')

Default directory: C:/Users/xsm/Desktop/MIEZE_data/


In [2]:
def new_reduce(subscans,print_fnames=False,print_raw=False,print_BGsub=False,tof=False,time_bins=32,clock_freq=100e3,\
               print_ROI=False,print_summed=False,ROI_x=(100,400),ROI_y=(100,400),direct=DEFAULT_DIRECT):
    """Returns the summed, background (BG) corrected, region of interest (ROI) 2d array of counts.
    The various print function optional arguments show the data processing step by step.
    The final image summed from the subscans is the last optional argument.
    Includes option for reducing time-dependent data."""
    assert isdir(direct), 'Data directory is not found!'
    
    filenames = [f'{direct}CG4B_{i}/sipm2d.dat' for i in subscans]
    for f in filenames:
        assert isfile(f), f'Datafile {f} was not found!'
    if print_fnames:
        print("Files used:",filenames)    
    ROIImages = []  #stores the background corrected region of interest
    for indx,filename in enumerate(filenames):
        if tof: raw_image = np.fromfile(filename,dtype=np.uint32).reshape(512,512,time_bins).T
        else: raw_image = np.fromfile(filename,dtype=np.uint32).reshape(512,512).T
        if print_raw:
            print('***Raw Image***')
            if tof: 
                max_indx = np.argmax(np.sum(raw_image,axis=(0,1)))
                min_indx = np.argmin(np.sum(raw_image,axis=(0,1)))
                print(f'Maximum counts: {max_indx}')
                ShowImage(raw_image[max_indx]) #only maximum and minimum counts
                print(f'Minimum counts: {min_indx}')
                ShowImage(raw_image[min_indx])
            else: ShowImage(raw_image)

        if tof: BG_pixels = raw_image[0:20:,0:20:,:]  #BG from all time bins
        else: BG_pixels = raw_image[0:20:,0:20:]
        BG_sum = np.sum(BG_pixels)
        BGperpixel =  float(BG_sum/np.size(BG_pixels))
        BGC_image = raw_image-BGperpixel  #subtracts background
        if print_BGsub:
            print("***BG subtracted image***")
            print("Subscan name: ",filename)
            print("BG per pixel:",BGperpixel)
            ShowImage(BGC_image)
        
        ROI_image = BGC_image[ROI_y[0]:ROI_y[1]:,ROI_x[0]:ROI_x[1]:]
        if print_ROI:
            print("***Region of interest***")
            print("Total counts in ROI: ",np.sum(ROI_image),"\n")
            ShowImage(ROI_image)

        ROIImages.append(ROI_image)

    summedROI = np.sum(ROIImages,axis=0)
    if print_summed:
        print("***Combined image of all subscans***")
        print("Total counts in summed ROI: ",np.sum(summedROI))
        ShowImage(summedROI)
    return summedROI

def ana_run(up,down,plot_pol=True,plot_slice=True,lims=(-1,1)):
    """Returns polarizations from reduced runs. Use down-count factor
    when count times are different for each spin state."""
    pol = (up-down)/(up+down)
    pol_x,pol_y = np.mean(pol,axis=0),np.mean(pol,axis=1)
    if plot_pol:
        plt.figure(figsize=(8,6))
        plt.imshow(pol,origin='lower',vmin=lims[0],vmax=lims[1])
        plt.colorbar()
        plt.tight_layout()
        plt.show()
    if plot_slice:
        plt.figure(figsize=(8,5))
        plt.plot(pol_x)
        plt.plot(pol_y)
        try: plt.ylim(lims)  #ignores limits
        except: pass
        plt.tight_layout()
        plt.show()
    return pol,pol_x,pol_y
    
def reduce_scan(scan_num_start,currents,ROI_x,ROI_y,ssn=10,plot_pol=False,\
                plot_slice=False,plot_all=True,plot_each=False,direct=DEFAULT_DIRECT):
    """Returns polarizations and intensities from an AER scan.
    snn is the number of subscans."""
    assert isdir(direct), 'Data directory is not found!'
    ints,intx,inty = [],[],[]
    pols,pxs,pys = [],[],[]
    for indx,c in enumerate(currents):
        subscans = range(scan_num_start+indx*ssn*2,scan_num_start+ssn+indx*ssn*2)
        ints.append(new_reduce(subscans,ROI_x=ROI_x,ROI_y=ROI_y,print_summed=False,direct=direct))
        intx.append(np.sum(ints[-1],axis=0))
        inty.append(np.sum(ints[-1],axis=1))
        subscans = range(scan_num_start+ssn+indx*ssn*2,scan_num_start+ssn*2+indx*ssn*2)
        ints.append(new_reduce(subscans,ROI_x=ROI_x,ROI_y=ROI_y,print_summed=False,direct=direct))
        intx.append(np.sum(ints[-1],axis=0))
        inty.append(np.sum(ints[-1],axis=1))
        p,px,py = ana_run(ints[-2],ints[-1],plot_pol=plot_pol,plot_slice=plot_slice)
        pols.append(p)
        pxs.append(px)
        pys.append(py)
    if plot_all:
        fig = plt.figure(figsize=(8,5))  #prints all polarizations
        for indx,p in enumerate(pxs):
            plt.plot(p,label=f'{round(currents[indx],3)}')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),handlelength=1)
        plt.ylim((-1,1))
        plt.tight_layout()
        plt.show()
    if plot_each:
        for indx,p in enumerate(pxs):
            fig = plt.figure(figsize=(8,5))  #prints each polarization separately
            plt.plot(p,label=f'{round(currents[indx],3)}')
            plt.legend()
            plt.ylim((-1,1))
            plt.tight_layout()
            plt.show()
        
    return np.array(ints),np.array(intx),np.array(inty),np.array(pols),np.array(pxs),np.array(pys)

In [ ]:
def get_scan(number):
    """Returns the data file scan number in Spice format."""
    const = 3
    if (number > 999):    #no leading zeros
        zeros = 0
    else:
        if (number == 0):
            zeros = const
        else:
            zeros = const - int(math.floor(math.log10(number)))
    return '0'*zeros + str(number)

default_order = ['det','det rate','mon rate','ac_1','ac_2','up_1','down_1','arm_2','detector','monitor','time','nu_1','nu_2','nu_3',\
                'rots','trans','sgl','sgu','stl','stu','a1','a2','focal_length','m1','m2','mfocus','cav_trans','s1','s2',\
                'q','h','k','l','ei','ef','e','nu_1_current','ac_1_current','prism_current','nu_2_current','ac_2_current','nu_3_current']

default_order = ['det','det rate','mon rate','ac_1','ac_2','up_1','down_1','up_2','down_2','detector','monitor','time',\
                'rots','trans','sgl','sgu','stl','stu','a1','a2','focal_length','m1','m2','mfocus','cav_trans','s1','s2',\
                'q','h','k','l','ei','ef','e','nu_1_current','ac_1_current','prism_current','nu_2_current','ac_2_current','nu_3_current']

default_order = ['det','det rate','mon rate','ac_1','ac_2','up_1','down_1','up_2','down_2','detector','monitor','time',\
                'cav_rot','cav_trans','sgl','sgu','stl','stu','a1','a2','focal_length','m1','m2','mfocus','cav_trans','s1','s2',\
                'q','h','k','l','ei','ef','e','nu_1_current','ac_1_current','prism_current','nu_2_current','ac_2_current','nu_3_current']


default_dict = {i[1]:i[0] for i in enumerate(default_order)}

useful_order = default_order[:12]
useful_dict = {i[1]:i[0] for i in enumerate(useful_order)}

def load_data(path, names, dest, write_data=False,overwrite=False,reorder=False):
    """Returns the list [scan number, scan date, scan time start, user input scan titles, matrix of data from scan].
    Cuts out all of the worthless parameters that we can't remove from SPICE for some reason."""
    scan_nums,dates,times,scan_titles,parms,data =[[],[],[],[],[],[]]
    for i in range(len(names)):
        file = open(path + names[i])
        lines = file.readlines()
        lines = [i.replace('\n','') for i in lines]    #removes new line characters
        file.close()
        
        scan_nums.append(lines[0][2::])    #line number determined by Spice format
        scan_nums[i] = scan_nums[i].split(' = ',)[1]
        dates.append(lines[1][2::])
        dates[i] = dates[i].split(' = ',)[1]
        times.append(lines[2][2::])
        times[i] = times[i].split(' = ',)[1]
        scan_titles.append(lines[10][2::])
        scan_titles[i] = scan_titles[i].split(' = ',)[1]
        parms.append(lines[29][7::].split())
        parms[i].append('det rate')
        parms[i].append('mon rate')
        data.append([j.split()[1::] for j in lines[30::] if (j[0] != '#')])    #assigns the data
        data[i] = [[float(data[i][j][k]) for j in range(len(data[i]))] for k in range(len(data[i][0]))]#HERE'S THE BUG    #converts to float
        data[i] = [list(j) for j in zip(*data[i])]    #transposes table
        
        time_normed_counts = [data[i][j][2]/data[i][j][1] if (data[i][j][1] != 0) else 0 for j in range(len((data[i])))]
        for j in range(len(data[i])):
            data[i][j].append(time_normed_counts[j])
            
        time_normed_counts2 = [data[i][j][3]/data[i][j][1] if (data[i][j][1] != 0) else 0 for j in range(len((data[i])))]
        for j in range(len(data[i])):
            data[i][j].append(time_normed_counts2[j])
        
        if (reorder and (parms[i] != default_order)):    #rearranges the columns to default_order
            swap_i = []    #columns to swap
            for j in range(len(parms[i])):
                swap_i.append([j, default_dict[parms[i][j]]])

            swaped_data = [ [0]*len(data[i][0]) for _ in range(len(data[i]))]    #matrix to hold swaped values
            for j in range(len(data[i])):    #slow swapping method, may need some work...
                for k in range(len(data[i][j])):
                    for m in range(len(swap_i)):
                        if (swap_i[m][1] == k):
                            swaped_data[j][k] = data[i][j][m]
                            parms[i][m] = default_order[swap_i[m][0]]    
            data[i] = swaped_data
            
        #useful parameters are hardcoded here
        try: data_out = np.array(data)[:,:,:12]
        except:
            print(f"Error with scan {scan_nums[i]}") 
            break
        parms_out = parms[0][:12]
        
        if write_data:    #writes data to csv files
            if overwrite: c_str = 'w'
            else: c_str = 'x'    #Fails if the file already exists
            
            windows_fname = scan_titles[i]    #Cleans the Spice file names for Windows
            invalid = '<>:"/\|?*'
            for char in invalid:
                windows_fname = windows_fname.replace(char,'')
                
            with open(dest_path + "scan" + str(scan_nums[i]) + "_" + windows_fname + ".csv", c_str, newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([scan_titles[i]])
                writer.writerow([dates[i]])
                writer.writerow([times[i]])
                writer.writerow(parms_out)
                for j in range(len(data_out[i])):
                    writer.writerow(data_out[i][j])
                    
    return scan_nums,dates,times,scan_titles,parms_out,data_out

data_path = "C://Users//samck//Desktop//SESANS_OAM_data//Datafiles"
dest_path = "C://Users//samck//Desktop//SESANS_OAM_data//data_csv//"

In [ ]:
def get_scan(number):
    """Returns the data file scan number in Spice format."""
    const = 3
    if (number > 999): zeros = 0
    else:
        if (number == 0): zeros = const
        else: zeros = const - int(np.floor(np.log10(number)))
    return '0'*zeros + str(number)

#parameters defined by SpICE initialization file
default_order = ['']
default_dict = {i[1]:i[0] for i in enumerate(default_order)}

useful_order = default_order[:]  #trims all unnecessary data from the SpICE datafile
useful_dict = {i[1]:i[0] for i in enumerate(useful_order)}

def data_loader(names,direct=DEFAULT_DIRECT,reorder=False):
    """Returns the list [scan number, scan date, scan time start, user input scan titles, matrix of data from scan].
    Cuts out all of the worthless parameters that we can't remove from SPICE for some reason."""
    assert isdir(direct), 'Data directory is not found!'
    scan_nums,dates,times,scan_titles,parms,data,data_out = [],[],[],[],[],[],[]  #This is a real cat's buffet
    for i in range(len(names)):
        file = open(direct + names[i])
        lines = file.readlines()
        lines = [i.replace('\n','') for i in lines]  #removes new line characters
        file.close()
        
        scan_nums.append(lines[0][2::])  #line number determined by Spice format
        scan_nums[i] = scan_nums[i].split(' = ',)[1]
        dates.append(lines[1][2::])
        dates[i] = dates[i].split(' = ',)[1]
        times.append(lines[2][2::])
        times[i] = times[i].split(' = ',)[1]
        scan_titles.append(lines[10][2::])
        scan_titles[i] = scan_titles[i].split(' = ',)[1]
        parms.append(lines[24][7::].split())

        data.append([j.split()[1::] for j in lines[25::] if (j[0] != '#')])  #assigns the data
        data[i] = [[float(data[i][j][k]) for j in range(len(data[i]))] for k in range(len(data[i][0]))]  #converts to float
        data[i] = [list(j) for j in zip(*data[i])]    #transposes table
        
        if (reorder and (parms[i] != default_order)):  #rearranges the columns to default_order
            swap_i = []    #columns to swap
            for j in range(len(parms[i])):
                swap_i.append([j, default_dict[parms[i][j]]])

            swaped_data = [ [0]*len(data[i][0]) for _ in range(len(data[i]))]  #matrix to hold swaped values
            for j in range(len(data[i])):    #slow swapping method, may need some work...
                for k in range(len(data[i][j])):
                    for m in range(len(swap_i)):
                        if (swap_i[m][1] == k):
                            swaped_data[j][k] = data[i][j][m]
                            parms[i][m] = default_order[swap_i[m][0]]    
            data[i] = swaped_data
            
        try: data_out.append(np.array(data[i][0])[:len(useful_order)])  #list of numpy arrays
        except: 
            data_out.append([])
            print(f"Error in processing scan {scan_nums[i]}!")
                
    if parms[0] == parms[-1]: parms_out = parms[0][:len(useful_order)]  #assumes parameter names are unchanged
    else: 
        parms_out = [parms[i][:len(useful_order)] for i in range(len(parms))]
        print('Parameter list changed!')
    try: data_out = np.array(data_out)
    except: print('Non-homogeneous data: unable to convert to numpy array!')
    
    return scan_nums,dates,times,scan_titles,parms_out,data_out

In [3]:
def ShowImage(inImage):
    """Plots image in reduction function."""
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    im = ax.imshow(inImage, origin="lower", aspect="auto")
    fig.colorbar(im)
    fig.tight_layout()
    plt.show()
    
def get_fit(pols,errs,cg_cs,ini_guess=[.1,210,0],plot=True,indxs=(50,55),p_sen=.5):
    """Returns the polarization fitted from phase scan. ini_guess is the initial guess at pixel 0.
    indxs controls which pixel fits to plot. p_sen sets the upper and lower bound for the phase fitting"""
    fitted,ferrs = [],[]
    cg_cs_fine = np.linspace(cg_cs[0],cg_cs[-1],200)
    
    fig = plt.figure(figsize=(8,6))
    phase = ini_guess[-1]  #intial phase guess
    for indx in range(len(pols[0,:])):
        points = pols[:,indx]
        point_errs = errs[:,indx]
        
        lb,ub = (0,ini_guess[1]-5,phase-p_sen),(1,ini_guess[1]+5,phase+p_sen)  #continuity of fitted phase

        guess=[min(np.max(points),.99),ini_guess[1],phase]
        parms,err = curve_fit(my_cos,cg_cs,points,p0=guess,bounds=(lb,ub),sigma=point_errs)
        fitted.append(parms)
        ferrs.append(np.sqrt(np.diag(err)))
        
        if indxs[0] < indx < indxs[1]:
            plt.plot(cg_cs,points,'o',color=f'C{indx%8}',label=f'{indx}')
            plt.plot(cg_cs_fine,my_cos(cg_cs_fine,*parms),color=f'C{indx%8}')
            if plot: print(f'amp,freq,phase = {np.round(parms,3)}')
        phase = parms[-1]  #continuity of fitted phase

    plt.legend()
    if plot: plt.show()
    else: plt.close()
    
    return np.array(fitted),np.array(ferrs)

def get_pol_errors(int_list,plot=False):
    """Returns polarization error."""
    ups = int_list[::2]
    dns = int_list[1::2]
    result = []
    for u,d in zip(ups,dns):
        temp = 2*np.sqrt(2)/(u+d)**2*np.sqrt(u**2*d+u*d**2)
        result.append(2*np.sqrt(2)/(u+d)**2*np.sqrt(u**2*d+u*d**2))
    if plot:
        plt.imshow(result,origin='lower',aspect='auto')
        plt.colorbar()
        plt.show()
    return np.nan_to_num(np.array(result),nan=.5)

def get_trad_error(empty_ints,sample_ints,plot=False):
    """Returns intensity error summed over phase scan."""
    empty = np.sum(empty_ints,axis=0)
    sample = np.sum(sample_ints,axis=0)
    #plt.plot(sample/empty)
    result = sample/empty*np.sqrt(1/empty+1/sample)
    if plot:
        plt.plot(result)
        plt.show()
    return np.nan_to_num(result,nan=np.max(result)*10)


def compare(e,err_e,s,err_s,plot_freq=False,plot_norm=False,plot_diff=True):
    """Plots the fitted parameters of blank and sample."""
    fig,ax = plt.subplots(1,2,figsize=(12,5))
    ax[0].set_title('Fitted amplitude')
    ax[0].plot(e[:,0],'.',label='empty',color='C0',ms=7.5,zorder=5,ls='-',linewidth=2.5)
    ax[0].errorbar(range(len(e[:,0])),e[:,0],err_e[:,0],ls='none',zorder=5,color='C0')
    ax[0].plot(s[:,0],'.',label='sample',color='C1',ms=7.5,zorder=5,ls='-',linewidth=2.5)
    ax[0].errorbar(range(len(s[:,0])),s[:,0],err_s[:,0],ls='none',zorder=5,color='C1')
    dark = s[:,0]/e[:,0]
    if plot_norm: ax[0].plot(dark,label='Normed',color='C2')
    ax[0].set_ylim((0,1))
    ax[0].legend()
    
    dark_err = dark*np.sqrt((err_e[:,0]/e[:,0])**2 + (err_s[:,0]/s[:,0])**2)
    
    ax[1].set_title('Fitted phase')
    ax[1].plot(e[:,2],'.',label='empty',color='C0',ms=7.5,zorder=5,ls='-',linewidth=2.5)
    ax[1].errorbar(range(len(e[:,2])),e[:,2],err_e[:,2],ls='none',zorder=5,color='C0')
    ax[1].plot(s[:,2],'.',label='sample',color='C1',ms=7.5,zorder=5,ls='-',linewidth=2.5)
    ax[1].errorbar(range(len(s[:,2])),s[:,2],err_s[:,2],ls='none',zorder=5,color='C1')
    phase = s[:,2]-e[:,2]
    if plot_diff: ax[1].plot(phase,label='Diff',color='C2')
    ax[1].legend()
    ax[1].set_ylim((np.min([np.min(e[:,2]),np.min(s[:,2])]),(np.max([np.max(e[:,2]),np.max(s[:,2])]))))
    plt.show()
    
    phase_err = err_e[:,2] + err_s[:,2]
    
    if plot_freq:
        fig = plt.figure(figsize=(8,6))
        plt.title('Fitted frequency')
        plt.plot(e[:,1],label='empty')
        plt.plot(s[:,1],label='sample')
        plt.legend()
        plt.show()
    return dark,dark_err,phase,phase_err

## Experimental parameters
***

## Data Analysis
***

### 8/29: Tuning and alignment